In [5]:
import json
#from sqlalchemy import create_engine
#import pymongo
import pandas as pd
#import avro

#HOME = os.environ["HOME"]
#sys.path.append(HOME)
#sys.path
#from ds_config import *


from pyspark import HiveContext

#from pyspark.sql import HiveContext

In [6]:
sqlc = HiveContext(sc)
sqlc.sql("SHOW TABLES").collect() 

[Row(result=u'ab_test_cases'),
 Row(result=u'ab_test_fires'),
 Row(result=u'ab_tests'),
 Row(result=u'academicdocuments'),
 Row(result=u'account_devices'),
 Row(result=u'account_drills'),
 Row(result=u'account_features'),
 Row(result=u'account_highlights'),
 Row(result=u'accounts'),
 Row(result=u'accounts_organizations'),
 Row(result=u'accounts_sponsors'),
 Row(result=u'accountstats'),
 Row(result=u'accountudids'),
 Row(result=u'achievementresults'),
 Row(result=u'active_coaches'),
 Row(result=u'activities'),
 Row(result=u'admin_messages'),
 Row(result=u'analytics'),
 Row(result=u'annotations'),
 Row(result=u'api_partners'),
 Row(result=u'api_partners_accounts'),
 Row(result=u'api_partners_videos'),
 Row(result=u'app_launches'),
 Row(result=u'athleteprofiles'),
 Row(result=u'athletes'),
 Row(result=u'athleteteamprofilepositions'),
 Row(result=u'athleteteamprofiles'),
 Row(result=u'auth_tokens'),
 Row(result=u'authenticationtokens'),
 Row(result=u'billingaccountinvoicelinking'),
 Row(re

In [16]:
#e = create_engine(mssql_conn)
##pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';", e)
#df = pd.read_sql("SELECT * FROM ContentServers;", e)

tmp = sqlc.sql("SELECT * FROM contentservers").map(lambda x: x.asDict()).collect()
#df.to_dict(outtype="records")
#contentServersRDD = sc.textFile("s3n://ds-fulla/sql/HuddleLite-150324T0322/ContentServers/")

#contentServersRDD.take(10)
urls_dict = { }
for x in tmp:
    urls_dict[x['serverid']] = x['httpreaduri']

sc_urls = sc.broadcast(urls_dict)


In [15]:
urls_dict

{1: u'http://edgecastcdn.net/0001EA/Thor/',
 2: u'http://hudlvideo01.s3.amazonaws.com/',
 3: u'http://va.hudl.com/',
 4: u'http://vb.hudl.com/',
 5: u'http://vc.hudl.com/',
 6: u'http://vd.hudl.com/',
 7: u'http://ve.hudl.com/',
 8: u'http://vf.hudl.com/',
 9: u'http://vg.hudl.com/',
 10: u'http://vh.hudl.com/',
 11: u'http://vi.hudl.com/',
 12: u'http://vj.hudl.com/',
 13: u'http://static.hudl.com/',
 14: u'http://vat.hudl.com/',
 41: u'http://vda.hudl.com/',
 42: u'http://vdb.hudl.com/',
 43: u'http://vdc.hudl.com/',
 44: u'http://vdd.hudl.com/',
 45: u'http://vde.hudl.com/',
 46: u'http://vcd.hudl.com/',
 47: u'http://v-arch.hudl.com/',
 102: u'http://hudlvideo01.s3.amazonaws.com/',
 103: u'http://va.hudl.com/',
 104: u'http://vb.hudl.com/',
 105: u'http://vc.hudl.com/',
 106: u'http://vd.hudl.com/',
 107: u'http://ve.hudl.com/',
 108: u'http://vf.hudl.com/',
 109: u'http://vg.hudl.com/',
 110: u'http://vh.hudl.com/',
 111: u'http://vi.hudl.com/',
 112: u'http://vj.hudl.com/',
 114:

In [17]:
sc.textFile("s3n://ds-fulla/mongo/20150323/monolith/coredata/clips/wUkTgF-zbe*.json")\
    .map(json.loads)\
    .take(10)

[{u'_id': 1309210964,
  u'a': [{u'an': u'Sideline',
    u'd': 1011010000,
    u'i': 1326135645,
    u'lt': u'kyb/kwz/6c4c8063-7c1d-4f72-994a-157f0bcedb73/038-vjjver6whciwwebw_l.jpg',
    u'mf': [{u'fn': u'kyb/kwz/6c4c8063-7c1d-4f72-994a-157f0bcedb73/vjjver6whciwwebw-3000.mp4',
      u'fs': 37977436,
      u'ft': 0,
      u'ha': False,
      u'q': 50,
      u'rv': {u'$binary': u'OhalBLcV0gg=', u'$type': u'00'},
      u's': 1},
     {u'fn': u'kyb/kwz/6c4c8063-7c1d-4f72-994a-157f0bcedb73/vjjver6whciwwebw-3000_1100.mp4',
      u'fs': 13934213,
      u'ft': 0,
      u'ha': False,
      u'q': 25,
      u'rv': {u'$binary': u'dHenBLcV0gg=', u'$type': u'00'},
      u's': 1}],
    u'mt': {u'$date': -62135596800000},
    u'o': 1,
    u'si': 112,
    u'th': u'kyb/kwz/6c4c8063-7c1d-4f72-994a-157f0bcedb73/038-vjjver6whciwwebw.jpg'}],
  u'cu': [{u'i': 16696307, u'o': 37}],
  u'd': {u'POSS #': 38},
  u'dc': {u'$date': 1423838779925},
  u'ec': False,
  u'pv': False,
  u's': 281364,
  u't': 16566},
 {u'

In [34]:
def getWide(x):
    retVal = [ ]
    for tmp in x:
        if "an" in tmp.keys():
            if tmp["an"]=="Wide":
                if "mf" in tmp.keys() and "si" in tmp.keys():
                    if tmp["mf"] != None:
                        for mydict in tmp["mf"]:
                            if "fn" in mydict:
                                if tmp["si"] in sc_urls.value:
                                    if sc_urls.value[tmp["si"]] != None:
                                        retVal.append( sc_urls.value[tmp["si"]] + mydict["fn"] )
    return retVal

In [35]:
#punt_keys = [ "PLAY TYPE", "PLAY", "OFF PLAY" ]
#field_goal_keys = [ "PLAY TYPE", "PLAY", "OFF PLAY" ]

def isType(x, play_type):
    possible_keys = [ "KICK TYPE", "PLAY TYPE", "PLAY", "OFF PLAY" ]
    for thekey in possible_keys:
        if thekey in x.keys():
            if play_type in str(x[thekey]).lower():
                return True
    return False

def punt_or_fg(x):
    if isType(x,"punt"):
        return "PUNT"
    elif isType(x,"fg"):
        return "FG"
    else:
        return ""

def getKeys(x):
    retVal = { }
    for mykey in x.keys():
        if "fg" in str(x[mykey]).lower():
            retVal[mykey] = x[mykey]
    return retVal


clipsRDD = sc.textFile("s3n://ds-fulla/mongo/20150323/monolith/coredata/clips/wUkTgF-zb*.json")\
    .map(json.loads)\
    .filter(lambda x: "d" in x.keys() and "a" in x.keys())\
    .map(lambda x: { "type" : punt_or_fg(x["d"]), "video" : getWide(x["a"]) })\
    .filter(lambda x: len(x["video"])>0)\
    .filter(lambda x: x["type"]!="")
    

In [36]:
clipsRDD.count()

59706

In [37]:
print clipsRDD.filter(lambda x: x["type"]=="FG").count()
print clipsRDD.filter(lambda x: x["type"]=="PUNT").count()

11143
48563


In [38]:
clipsRDD.saveAsTextFile("s3n://hudl-hadoop/GLaDOS/fg_punt_locations/")

In [39]:
clipsRDD.take(100)

[{'type': 'PUNT',
  'video': [u'http://vg.hudl.com/lom/chr/5dec0480-7145-4641-af87-7290bdb9559c/ipvht6jcxo8qld17-1100.mp4']},
 {'type': 'PUNT',
  'video': [u'http://vg.hudl.com/hgy/vs9/2998d26f-d8b2-4e9e-9dc4-ef74c7140a09/ejduixvtkmn3lw7v-1100.mp4']},
 {'type': 'PUNT',
  'video': [u'http://vg.hudl.com/hgy/vs9/2998d26f-d8b2-4e9e-9dc4-ef74c7140a09/bebmsnqr1evgevzf-1100.mp4']},
 {'type': 'PUNT',
  'video': [u'http://vg.hudl.com/hgy/vs9/2998d26f-d8b2-4e9e-9dc4-ef74c7140a09/xndlzaedavqm6qqm-1100.mp4']},
 {'type': 'PUNT',
  'video': [u'http://vg.hudl.com/hgy/vs9/2998d26f-d8b2-4e9e-9dc4-ef74c7140a09/n1ndgqqilkwyyubl-1100.mp4']},
 {'type': 'PUNT',
  'video': [u'http://vg.hudl.com/hgy/vs9/2998d26f-d8b2-4e9e-9dc4-ef74c7140a09/vnomxjo2lwkkiw7w-1100.mp4']},
 {'type': 'PUNT',
  'video': [u'http://vg.hudl.com/lom/chr/5dec0480-7145-4641-af87-7290bdb9559c/mlher9m70dhpf6bn-1100.mp4']},
 {'type': 'PUNT',
  'video': [u'http://vb.hudl.com/agi/cc6/d55c2198-0e06-4631-9c5f-542cfe4d11e7/onmvtjj5w0fic7gr-3000.